In [1]:
!pip install pyswarms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
max_bound = [5.12, 5.12]
min_bound = [-5.12, -5.12]
bounds = (min_bound, max_bound)
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, bounds=bounds,
options=options)
optimizer.optimize(fx.rastrigin, iters=1000)

2025-06-12 12:09:45,818 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0
2025-06-12 12:09:47,715 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [1.37306381e-09 5.19438517e-10]


(np.float64(0.0), array([1.37306381e-09, 5.19438517e-10]))

In [3]:
def cost_function(I):
  U = 10
  R = 100
  I_s = 9.4e-12
  v_t = 25.85e-3
  c = abs(U - v_t * np.log(abs(I[:,0] / I_s)) - R * I[:,0])
  return c
options = {'c1':0.5,'c2':0.3,'w':0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=1, options=options)
cost, pos = optimizer.optimize(cost_function, iters=200)

2025-06-12 12:09:51,850 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|200/200, best_cost=8.53e-5
2025-06-12 12:09:52,487 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 8.526476538506245e-05, best pos: [0.09404684]


In [4]:
from scipy.optimize import fsolve
def cost(I):
 return abs(10-25.85e-3* np.log(abs(I /9.4e-12)) -100* I)
initial_guess = 0.09
I = fsolve(func=cost, x0=initial_guess)
print(I[0])

0.09404768643017938


In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from keras import losses

# Učitavanje podataka
data = load_iris()
X = data.data
y = data.target

# One-hot enkodiranje targeta
y_new = np.zeros((X.shape[0], 3))
for i in range(X.shape[0]):
    y_new[i, y[i]] = 1.
y = y_new

# Podjela na trening i test skup
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Parametri mreže
n_inputs = 4
n_hidden = 20
n_classes = 3
num_samples = 150

# Funkcija za izračun vjerovatnoća
def probs_function(p, X):
    W1 = p[0:80].reshape((n_inputs, n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden, n_classes))
    b2 = p[160:163].reshape((n_classes,))
    z1 = X.dot(W1) + b1
    a1 = np.tanh(z1)
    logits = a1.dot(W2) + b2
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return probs

# Funkcija za računanje gubitka (loss)
def forward_prop(params):
    probs = probs_function(params, X_train)
    cce = losses.CategoricalCrossentropy()
    loss = cce(y_train, probs).numpy()
    return loss


2025-06-12 12:10:00,127 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


In [6]:
def f(x):
  n_particles = x.shape[0]
  values = [forward_prop(x[i]) for i in range(n_particles)]
  return np.array(values)


In [7]:
%%time
options = {'c1':0.5, 'c2':0.3, 'w':0.9}
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)
cost, pos = optimizer.optimize(f, iters=100)

2025-06-12 12:10:08,062 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.0572
2025-06-12 12:10:37,943 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.057206761091947556, best pos: [ 2.19538786e+00 -3.93844438e-01 -9.56364226e-01  1.10733197e+00
  1.27375885e+00  9.80652126e-01 -8.84298070e-01  1.15295727e+00
  8.82008990e-01  4.10215029e-01  9.52782321e-01 -3.47886380e-01
  4.13655640e-01 -1.54008772e+00  1.04477139e+00  6.35332121e-01
  5.84037336e-01 -2.09730585e+00 -1.38866616e+00 -8.03101100e-01
 -1.73361829e+00  4.54628858e-01 -6.18786405e-01 -1.77144288e+00
  1.32281962e+00 -9.42390630e-01  8.31877428e-01  4.77406351e-01
  2.78135393e+00  2.94954102e-01  2.49321544e+00  1.88875725e-01
  6.62693527e-03 -3.35121760e-01 -4.51380926e-01 -7.30249881e-01
  1.64205153e+00 -3.05298281e-01 -6.82506802e-01 -7.16352958e-01
 -2.71988753e-01  1.95113

CPU times: user 29.3 s, sys: 173 ms, total: 29.5 s
Wall time: 29.9 s


In [8]:
def predict(pos):
  probs = probs_function(pos, X_test)
  return probs


(np.round(predict(pos)).astype('int') == y_test).mean()

np.float64(1.0)